# 2022 NZMSA Data Science Submission.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

# Exploratory Data Analysis.

## Numerical

In [ ]:
dataset = pd.read_csv("weather-data.csv")

print("Average (mean):")
print(dataset[['tmpc', 'dwpc', 'relh', 'sknt', 'gust']].mean())
print("-"*20)
print("Standard deviation:")
print(dataset[['tmpc', 'dwpc', 'relh', 'sknt', 'gust']].std())
print("-"*20)
print("10th and 90th percentile range")
percentiles_10th = dataset[['tmpc', 'dwpc', 'relh', 'sknt', 'gust']].quantile(0.1)
percentiles_90th = dataset[['tmpc', 'dwpc', 'relh', 'sknt', 'gust']].quantile(0.9)
print(percentiles_90th - percentiles_10th)

## Graphical

In [ ]:
# correlation plot
corrMatrix = dataset[['tmpc', 'dwpc', 'relh', 'sknt', 'gust']].dropna().corr()
sns.heatmap(corrMatrix, annot=True)
plt.title("Correlation matrix of weather data")
plt.show()

# plot all numerical data
dataset['valid'] = pd.to_datetime(dataset['valid'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce') # convert valid dates to datetime objects
plt.style.use("bmh")
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5, ncols=1, squeeze=True)

tmpc_data = dataset[['valid','tmpc']].dropna()
ax1.plot(tmpc_data["valid"], tmpc_data["tmpc"], color="blue")
ax1.set_title("Environment Temperature")
ax1.set_xlabel("Time")
ax1.set_ylabel("Environment Temperature (°C)")

dwpc_data = dataset[['valid','dwpc']].dropna()
ax2.plot(dwpc_data["valid"], dwpc_data["dwpc"], color="red")
ax2.set_title("Dew point Temperature")
ax2.set_xlabel("Time")
ax2.set_ylabel("Dew point Temperature (°C)")

relh_data = dataset[['valid','relh']].dropna()
ax3.plot(relh_data["valid"], relh_data["relh"], color="green")
ax3.set_title("Relative humidity")
ax3.set_xlabel("Time")
ax3.set_ylabel("Relative humidity (%)")

sknt_data = dataset[['valid','sknt']].dropna()
ax4.plot(sknt_data["valid"], sknt_data["sknt"], color="orange")
ax4.set_title("Wind speed")
ax4.set_xlabel("Time")
ax4.set_ylabel("Wind speed (knots)")

gust_data = dataset[['valid','gust']].dropna()
ax5.plot(gust_data["valid"], gust_data["gust"], color="black")
ax5.set_title("Wind gust")
ax5.set_xlabel("Time")
ax5.set_ylabel("Wind gust (knots)")

fig.set_size_inches(10,25)

## Comment on the dataset

All the data in the dataset appear to be collected from the same station which is NZAA. The data are collected on the dates ranging from 1/01/2015 0:00 to 30/12/2020 23:30. The the gust measurement column of the dataset is sparsely populated compared to the other measurement columns of tmpc, dwpc, relh and sknt. The peak_wind_drct column appears to have no data present in it so it was ignored in the EDA. For some tmpc and dwpc, there is no available data for some dates.

# Data Processing

## Making changes

In [ ]:
dataset = pd.read_csv("weather-data.csv")
print(dataset.head()) # Show the first five results.

# remove useless data
dataset.drop(columns=[" station", "gust", "peak_wind_drct"], inplace=True)
dataset = dataset[['valid', 'tmpc']]
dataset.dropna(inplace=True)
dataset['series'] = np.arange(1,len(dataset)+1)

# convert 'valid' column data type to datetime and extract year and month
dataset['valid'] = pd.to_datetime(dataset['valid'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
dataset['year'] =   [i.year for i in dataset['valid']]
dataset['month'] =   [i.month for i in dataset['valid']]
dataset.drop(['valid'], axis=1, inplace=True)
dataset

In [ ]:
# split data into train-test set
# test data will be all data starting from start of 2020 and training data is all data prior to that
train = dataset[dataset['year'] < 2020]
test = dataset[dataset['year'] >= 2020]

fig, ax=plt.subplots(figsize=(9, 4))
train['tmpc'].plot(ax=ax, label='train')
test['tmpc'].plot(ax=ax, label='test')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Environment Temperature (°C)')
plt.title("Train test split")
plt.show()

## Reason for changes
- ` station` column is removed because they are all from NZAA
- `gust` column is removed because there is not sufficient data matching with other data
    - only using other data where gust was recorded as well will waste much of the data we have
- `peak_wind_drct` column is removed since there is no data in this column
- `valid` column converted to datetime format to extract the year and month numerical data
    - `valid` was previously in object format which is not compatible with models
- The dataset is reduced to the columns `valid` and `tmpc` to be used in the model
- `series` column is added for future predictions

# Data Modelling

In [ ]:
# Plot
# ==============================================================================
fig, ax = plt.subplots(figsize=(9, 4))
data_train['tmpc'].plot(ax=ax, label='train')
data_test['tmpc'].plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predictions')
plt.legend()
plt.show()

# Conclusion

* Comment on your model performance.
* Anything interesting about the data analysis.